In [8]:
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import time

from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader, sampler
from torch import nn
from train import acc_metric, dice_metric, get_random_folder_split
from DatasetMedical import DatasetCAMUS_r, DatasetCAMUS
from Unet2D import Unet2D
from torchvision import transforms

In [2]:
def test(model, test_dl, acc_fn, dice_fn):
    model.cuda()
    model.eval()
    running_acc = 0
    running_dice = 0
    for x, y in test_dl:
        x = x.cuda()
        y = y.cuda()
        with torch.no_grad():
            outputs = model(x)
            acc = acc_fn(outputs, y)
            dice = dice_fn(outputs, y)
        running_acc += acc
        running_dice += dice
    
    return running_acc/len(test_dl), running_dice/len(test_dl)

In [3]:
def main(params_path):
    bs=1
    
    preprocess = transforms.Compose([
        #transforms.GaussianBlur(3, sigma=0.1)
        #transforms.Resize((224,224))
    ])
    
    # datasets, 0=background+LV, 1=b+LV+M+RV, 2=??
    datasets = ['CAMUS_resized', 'CAMUS', 'TEE']
    curr_dataset = datasets[1]
    
    # datasets
    if curr_dataset == 'CAMUS_resized':
        base_path = Path('/work/datasets/medical_project/CAMUS_resized')
        _, _, test_files = get_random_folder_split(base_path)
       test_dataset = DatasetMedical(base_path / 'train_gray', test_files,
                                     base_path / 'train_gt')
    elif curr_dataset == 'CAMUS':
        base_path = Path('/work/datasets/medical_project/CAMUS')
        _, _, test_files = get_random_folder_split(base_path)
        test_dataset = DatasetMedical(base_path / 'train_gray', test_files,
                                     base_path / 'train_gt')
    elif curr_dataset == 'TEE':
        base_path = Path('/work/datasets/medical_project/TEE')
        _, _, test_files = get_random_folder_split(base_path)
       test_dataset = DatasetMedical(base_path / 'train_gray', test_files,
                                      base_path / 'train_gt')
    print(len(data))

    # split the training dataset and initialize the data loaders
    test_dl = DataLoader(test_dataset, batch_size=bs, shuffle=False)
    print(len(test_dl))
    xb, yb = next(iter(test_dl))
    print(xb.shape, yb.shape)

    # build the Unet2D with one channel as input and 2 channels as output
    unet = Unet2D(1, 2)
    # Load best model params
    unet.load_state_dict(torch.load(params_path))
    
    acc, dice = test(unet, test_dl, acc_metric, dice_metric)
    print('acc: ', acc,'dice: ', dice)

In [4]:
params_path = "models/model_epoch_final.pth"
main(params_path)

patient0004_4CH_ED.mhd loaded. Shape: (641, 908, 1)
patient0004_4CH_ED_gt.mhd loaded. Shape: (641, 908, 1)
patient0134_4CH_ED.mhd loaded. Shape: (482, 843, 1)
patient0134_4CH_ED_gt.mhd loaded. Shape: (482, 843, 1)
patient0008_4CH_ED.mhd loaded. Shape: (787, 1297, 1)
patient0008_4CH_ED_gt.mhd loaded. Shape: (787, 1297, 1)
patient0017_4CH_ED.mhd loaded. Shape: (630, 1038, 1)
patient0017_4CH_ED_gt.mhd loaded. Shape: (630, 1038, 1)
patient0411_4CH_ED.mhd loaded. Shape: (512, 843, 1)
patient0411_4CH_ED_gt.mhd loaded. Shape: (512, 843, 1)
patient0399_4CH_ED.mhd loaded. Shape: (624, 973, 1)
patient0399_4CH_ED_gt.mhd loaded. Shape: (624, 973, 1)
patient0011_4CH_ED.mhd loaded. Shape: (512, 843, 1)
patient0011_4CH_ED_gt.mhd loaded. Shape: (512, 843, 1)
patient0067_4CH_ED.mhd loaded. Shape: (551, 908, 1)
patient0067_4CH_ED_gt.mhd loaded. Shape: (551, 908, 1)
patient0151_4CH_ED.mhd loaded. Shape: (708, 1167, 1)
patient0151_4CH_ED_gt.mhd loaded. Shape: (708, 1167, 1)
patient0108_4CH_ED.mhd loaded. 

68
torch.Size([1, 1, 384, 384]) torch.Size([1, 384, 384])
acc:  tensor(1., device='cuda:0') dice:  tensor(nan, device='cuda:0')
